In [ ]:
import pandas as pd
import re
import math
import requests
import json

In [ ]:
df = pd.read_csv('phishing_site_urls.csv')
df.head()

In [ ]:
# @Globals
tldSet = set()
bad_url_word_frequency = {}
good_url_word_frequency = {}
sensitive_words = []

In [ ]:
def count_digits(word):
  return len(re.findall('[0-9]', word))

def count_alphabets(word):
  return len(re.findall('[A-z]', word))

In [ ]:
def remove_special_characters(input_string):
  # Pattern to match any non-alphanumeric and non-space character
  pattern = r'[^a-zA-Z0-9\s]'  
  return re.sub(pattern, '', input_string)

In [ ]:
def calculate_entropy(word):
  if len(word) == 0:
    return 0

  entropy = 0
  word_length = len(word)
  char_count = {}

  for char in word:
    if char in char_count:
      char_count[char] += 1
    else:
      char_count[char] = 1

  for char in char_count:
    probability = char_count[char] / word_length
    entropy += -probability * math.log2(probability)

  return entropy

In [ ]:
def count_special_characters(input_string):
  special_characters = "!@#$%^&*()-_=+[]{}|;:',.<>/?"
  count = 0

  for char in input_string:
    if char not in special_characters and not char.isalnum():
      count += 1

  return count

In [ ]:
# https://www.whoisxmlapi.com/whoisserver/WhoisService?apiKey=at_BGLU4RpM0fcKHpqsqHAYRWFl4bIt2&domainName=facebook.com
# Rapid API Key: d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e (Shauryansingh23102001@gmail.com)

# FUNCTION TO MAKE REQUEST TO WHOISAPI THROUGH RAPIDAPI (Not in use currently)
def whois_api_request(api_key, domain_url):
    url = "https://whoisapi-whois-v2-v1.p.rapidapi.com/whoisserver/WhoisService"

    headers = {
	    "X-RapidAPI-Key": "d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e",
	    "X-RapidAPI-Host": "whoisapi-whois-v2-v1.p.rapidapi.com"
    }

    querystring = {
        "domainName":domain_url,
        "apiKey":api_key,
        "outputFormat":"JSON",
        "da":"0",
        "ipwhois":"1",
        "thinWhois":"0",
        "_parse":"0",
        "preferfresh":"1",
        "checkproxydata":"0",
        "ip":"1"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    
    return response

In [ ]:
"""
for (index, colname) in enumerate(df):
  if(index == 2500):
    break

  URL = df["URL"].values
  domain = URL.split("/")[0]
  url_length = len(URL)
  domain_entropy = calculate_entropy(domain)

  # Top Level Domain Extraction Logic
  tld = domain.split(".")[-1]
  tld_length = len(tld)
  tld = tld.split(":")[0]
  tld = tld.split("-")[0]
  if(tld_length > 1 and tld_length < 10 and count_alphabets(tld) > count_digits(tld)):
    tldSet.add(tld)

  if df["Label"].values == "bad":
    temp = URL.split("/")
    subdomain = "/".join(temp[1:])
    words_list = re.split("[/,.,?,-,+,=,&,~,!,@,#,$,%,_,;]", subdomain)
    for word in words_list:
      word = word.strip().lower()
      word = remove_special_characters(word)
      digs = len(re.findall('[0-9]', word))
      alps = len(re.findall('[A-z]', word))
      if(len(word) < 2 or len(word) > 15 or digs > alps):
        continue
      if word in bad_url_word_frequency:
        bad_url_word_frequency[word] += 1
      else:
        bad_url_word_frequency[word] = 1
  else:
    temp = URL.split("/")
    subdomain = "/".join(temp[1:])
    words_list = re.split("[/,.,?,-,+,=,&,~,!,@,#,$,%,_,;]", subdomain)
    for word in words_list:
      word = word.strip().lower()
      word = remove_special_characters(word)
      digs = len(re.findall('[0-9]', word))
      alps = len(re.findall('[A-z]', word))
      if(len(word) < 2 or len(word) > 15 or digs > alps):
        continue
      if word in good_url_word_frequency:
        good_url_word_frequency[word] += 1
      else:
        good_url_word_frequency[word] = 1

print(len(good_url_word_frequency))
print(len(bad_url_word_frequency))
"""

In [ ]:
#LAST
start_row_idx = 3522
end_row_idx = 4002

# TO DO
start_row_idx = 4003
end_row_idx = 4483

df = df.iloc[start_row_idx : end_row_idx]
print(df.shape[0])
print(df.head())

In [ ]:
print(df.tail())

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
print(df.head())

In [ ]:
# UPDATED WHOIS API FUNCTION

def get_whois_data(api_key, domain_name):
  base_url = "https://www.whoisxmlapi.com/whoisserver/WhoisService"
  params = {
    "apiKey": api_key,
    "domainName": domain_name,
    "outputFormat":"JSON",
    "preferfresh":"1",
  }

  response = requests.get(base_url, params=params)
  return response

In [ ]:
# NEW ONE
# Index Starts from 1
DATA = []

# START = start_row_idx - 1
cnt = 0
for index, row in df.iterrows():
  if(cnt > 500):
    break
    
  URL = row["URL"]
  label = row["Label"]

  domain = URL.split("/")[0]
  url_length = len(URL)
  domain_entropy = calculate_entropy(domain)

  temp = URL.split("/")
  subdomain = temp[0].split(".")[-1]
  path_rest = "/".join(temp[1:])
  path_rest_length = len(path_rest)

  words_list = re.split("[/,.,?,-,+,=,&,~,!,@,#,$,%,_,;]", path_rest)

  tld = domain.split(".")[-1]
  tld_length = len(tld)
  tld = tld.split(":")[0]
  tld = tld.split("-")[0]

  num_spcs_chars = count_special_characters(path_rest)

  url = "https://whoisapi-whois-v2-v1.p.rapidapi.com/whoisserver/WhoisService"

  querystring = {
    "domainName":domain,
    "apiKey":"at_u6XSMmxG3lpSkcRJFsHzt60WqQQUc",
    "outputFormat":"JSON",
    "da":"0",
    "ipwhois":"1",
    "thinWhois":"0",
    "_parse":"0",
    "preferfresh":"1",
    "checkproxydata":"0",
    "ip":"1"
  }

  headers = {
    "X-RapidAPI-Key": "d2592fe48dmsh190428470dadf9cp1bf762jsnf40816e6115e",
    "X-RapidAPI-Host": "whoisapi-whois-v2-v1.p.rapidapi.com"
  }
  
  response = requests.get(url, headers=headers, params=querystring)
  
  response_data = response.json()
  
  if "ErrorMessage" in response_data:
    continue

  if "WhoisRecord" in response_data:
    if "createdDate" in response_data["WhoisRecord"]:
      created_year = response_data["WhoisRecord"]["createdDate"][0:4]
    elif "registryData" in response_data["WhoisRecord"] and "createdDate" in response_data["WhoisRecord"]["registryData"]:
      created_year = response_data["WhoisRecord"]["registryData"]["createdDate"][0:4]
    else:
      created_year = None

  if "WhoisRecord" in response_data:
    if "updatedDate" in response_data["WhoisRecord"]:
      updated_year = response_data["WhoisRecord"]["updatedDate"][0:4]
    elif "registryData" in response_data["WhoisRecord"] and "updatedDate" in response_data["WhoisRecord"]["registryData"]:
      updated_year = response_data["WhoisRecord"]["registryData"]["updatedDate"][0:4]
    else:
      updated_year = None

  if "WhoisRecord" in response_data:
    if "expiresDate" in response_data["WhoisRecord"]:
      expires_year = response_data["WhoisRecord"]["expiresDate"][0:4]
    elif "registryData" in response_data["WhoisRecord"] and "expiresDate" in response_data["WhoisRecord"]["registryData"]:
      expires_year = response_data["WhoisRecord"]["registryData"]["expiresDate"][0:4]
    else:
      expires_year = None

  if "WhoisRecord" in response_data:
    if ("registrant" in response_data["WhoisRecord"] and 
      "countryCode" in response_data["WhoisRecord"]["registrant"]):
      country_code = response_data["WhoisRecord"]["registrant"]["countryCode"]
    elif (
      "registryData" in response_data["WhoisRecord"] and
      "registrant" in response_data["WhoisRecord"]["registryData"] and
      "countryCode" in response_data["WhoisRecord"]["registryData"]["registrant"]):
      country_code = response_data["WhoisRecord"]["registryData"]["registrant"]["countryCode"]
    else:
      country_code = None
      
  if "WhoisRecord" in response_data and "estimatedDomainAge" in response_data["WhoisRecord"]:
    domain_age = response_data["WhoisRecord"]["estimatedDomainAge"]
  else:
    domain_age = None

  df2_data = [subdomain, domain, tld, path_rest, url_length, path_rest_length, num_spcs_chars, domain_entropy, domain_age, created_year, updated_year, expires_year, country_code]

  cnt += 1
  DATA.append(df2_data)

In [ ]:
df2 = pd.DataFrame(DATA, columns=["subdomain", "domain", "tld", "path_rest", "url_length", "path_rest_length", "num_spcs_chars", "domain_entropy", "domain_age", "created_year", "updated_year", "expires_year", "country_code"])

In [ ]:
print(DATA)

In [ ]:
df2.to_csv('df2.csv', index=False)

In [ ]:
# df2 = df2.reindex(df.index)

In [ ]:
# INSERT COLUMNS IN BETWEEN THE DATAFRAME
insert_position = df.columns.get_loc('URL') + 1

for col_name in df2.columns[::-1]:
  df.insert(insert_position, col_name, df2[col_name])

In [ ]:
df.to_csv('newData/url_4003_to_4483.csv', index=False)

In [ ]:
""" # Iterate all rows using DataFrame.iterrows() 
for index, row in df.iterrows():
  print(index)
  if(index == 25):
    break

  URL = row["URL"]
  label = row["Label"]
  domain = URL.split("/")[0]
  url_length = len(URL)
  domain_entropy = calculate_entropy(domain)

  # Top Level Domain Extraction Logic
  tld = domain.split(".")[-1]
  tld_length = len(tld)
  tld = tld.split(":")[0]
  tld = tld.split("-")[0]
  if(tld_length > 1 and tld_length < 10 and count_alphabets(tld) > count_digits(tld)):
    tldSet.add(tld)

  temp = URL.split("/")
  subdomain = "/".join(temp[1:])
  words_list = re.split("[/,.,?,-,+,=,&,~,!,@,#,$,%,_,;]", subdomain)
  
  # Create Word Frequency for Good and Bad URLs
  if label == "bad":
    for word in words_list:
      word = word.strip().lower()
      word = remove_special_characters(word)
      digs = len(re.findall('[0-9]', word))
      alps = len(re.findall('[A-z]', word))
      
      if(len(word) > 2 or len(word) < 15 or digs < alps):  
        if word in bad_url_word_frequency:
          bad_url_word_frequency[word] += 1
        else:
          bad_url_word_frequency[word] = 1
  else:
    for word in words_list:
      word = word.strip().lower()
      word = remove_special_characters(word)
      digs = len(re.findall('[0-9]', word))
      alps = len(re.findall('[A-z]', word))
      
      if(len(word) > 2 or len(word) < 15 or digs < alps):
        if word in good_url_word_frequency:
          good_url_word_frequency[word] += 1
        else:
          good_url_word_frequency[word] = 1

sorted_good_url_word_frequency = sorted(good_url_word_frequency.items(), key=lambda x:x[1], reverse=True)
sorted_bad_url_word_frequency = sorted(bad_url_word_frequency.items(), key=lambda x:x[1], reverse=True) """

In [ ]:
for it in good_url_word_frequency:
  print(it)

In [ ]:
for it in bad_url_word_frequency:
    print(it)

In [ ]:
for it in good_url_word_frequency:
  if(it[1] > 5):
    sensitive_words.append(it)

In [ ]:
for it in bad_url_word_frequency:
  if(it[1] > 5):
    sensitive_words.append(it)

#### *** Export Sensitive Words to a file. Only the words sensitive_words[0] *** ###
#### *** Loop through good label rows to export their TLD to a file *** ###

In [ ]:
with open('top_level_domains.txt', 'w', encoding='utf-8') as file:
    for item in tldSet:
        file.write(str(item) + '\n')

In [ ]:
with open('output.txt', 'w') as file:
  for item in sensitive_words:
    file.write(str(item) + '\n')